In [2]:
_RIOT_API_KEY='RGAPI-eb715053-5bb4-49af-b624-970e0673d75f'
from riotwatcher import LolWatcher
import helper_functions as help
import constants
import pandas as pd
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

lol_watcher = LolWatcher(_RIOT_API_KEY)

In [3]:
#match = lol_watcher.match.by_id('na1', 'NA1_3891774034')
#match = lol_watcher.match.by_id('na1', 'NA1_4720874733')
engine = help.create_mysql_engine()

# Plan for New Role Classification
1. Create new DB table 'lol_match_roles' with: matchId, participantId, Role, validRoles
2. Re-Create function for Jungle Flag
2. Re-Create function for Support Flag
3. Re-Create function for Lane By Posx/Posy
4. Create Mother Function To Validate positions for a match. validRoles = True if 1 player in each role, False otherwise. Upload to new 
5. Probably create a 'lol_analysis_view' which pulls needed info from 'lol_match_details', 'lol_match_timeline', and 'lol_match_roles' which an analysis script can pull from.


In [4]:
def classify_jungler(match_list_to_classify, db_engine):
    
    query=f"""  SELECT 
                    md.matchid,
                    md.participantid,
                    md.neutralMinionsKilled as totaljungleMinionsKilled,
                    lower(concat( item0Name, ' ', item1Name, ' ',  item2Name, ' ',  item3Name, ' ', 
                            item4Name, ' ',  item5Name, ' ',  item6Name )) as itemNames,
                    lower(concat( summoner1Name,  ' ', summoner2Name )) as summonerSpells
                FROM riot_api.lol_match_details md 
                where md.matchId in {tuple(match_list_to_classify)} """
    df = pd.read_sql(query, con=engine)
    
    df['jungle_flag'] = [True if 'smite' in spells else False  for spells in df.summonerSpells ]
    num_junglers = df.groupby('matchid')['jungle_flag'].sum().reset_index().rename(columns={'jungle_flag': 'num_junglers'},inplace=False)
    df2 = df.merge(num_junglers, left_on='matchid', right_on='matchid')

    
    matches_w_too_many_junglers = df[(df2['jungle_flag']==True) & (df2['num_junglers'] > 2)].matchid.unique()
    print (f'{len(matches_w_too_many_junglers)} matches with more than 2 smites:')
    print(matches_w_too_many_junglers)

    for matchid in matches_w_too_many_junglers:
        for index in df2[(df2['matchid'] == matchid) &(df2['jungle_flag']==True)].sort_values(by='totaljungleMinionsKilled', ascending=False).index[2:]:
            df2.loc[index,'jungle_flag'] = False

    return df2[['matchid','participantid','jungle_flag']]


to_classify = help.find_to_classify(engine)[:2000]
#print(to_classify)
classify_jungler(to_classify, engine)

TypeError: argument of type 'NoneType' is not iterable

In [11]:
num_to_classify = 10
to_classify = help.find_to_classify(engine)
tuple(to_classify[:num_to_classify])


('NA1_4720874733',
 'NA1_4716080477',
 'NA1_4716036182',
 'NA1_4716002274',
 'NA1_4711290187',
 'NA1_4711257700',
 'NA1_4711215827',
 'NA1_4711172258',
 'NA1_4710238204',
 'NA1_4710191130')

In [90]:
query=f""" SELECT 
md.matchid,
md.participantid,
md.neutralMinionsKilled as totaljungleMinionsKilled,
lower(concat( item0Name, ' ', item1Name, ' ',  item2Name, ' ',  item3Name, ' ', 
        item4Name, ' ',  item5Name, ' ',  item6Name )) as itemNames,
lower(concat( summoner1Name,  ' ', summoner2Name )) as summonerSpells
FROM riot_api.lol_match_details md 
 where md.matchid in ('NA1_3908483015',
'NA1_3797672185',
'NA1_4629886434')"""
#where md.matchId in {tuple(to_classify[:num_to_classify])} """
df = pd.read_sql(query, con=engine)
df

,matchid,participantid,totaljungleMinionsKilled,itemNames,summonerSpells
0,NA1_4629886434,1,0,None,teleport flash
1,NA1_4629886434,2,148,"jak'sho, the protean demonic embrace sorcerer'...",flash smite
2,NA1_4629886434,3,5,None,teleport flash
3,NA1_4629886434,4,3,galeforce berserker's greaves lord dominik's r...,heal flash
4,NA1_4629886434,5,0,None,exhaust flash
5,NA1_4629886434,6,10,anathema's chains sunfire aegis spirit visage ...,flash teleport
6,NA1_4629886434,7,146,None,flash smite
7,NA1_4629886434,8,20,wit's end death's dance zhonya's hourglass pla...,ignite flash
8,NA1_4629886434,9,28,rapid firecannon refillable potion kraken slay...,heal flash
9,NA1_4629886434,10,5,bulwark of the mountain mercury's treads chain...,ghost flash


In [92]:
df['jungle_flag'] = [True if 'smite' in spells else False  for spells in df.summonerSpells ]
num_junglers = df.groupby('matchid')['jungle_flag'].sum().reset_index().rename(columns={'jungle_flag': 'num_junglers'},inplace=False)
df2 = df.merge(num_junglers, left_on='matchid', right_on='matchid')


In [93]:
for matchid in df[(df2['jungle_flag']==True) & (df2['num_junglers'] > 2)].matchid.unique():
    display(df2[(df2['matchid'] == matchid) &(df2['jungle_flag']==True)].sort_values(by='totaljungleMinionsKilled', ascending=False))
    for index in df2[(df2['matchid'] == matchid) &(df2['jungle_flag']==True)].sort_values(by='totaljungleMinionsKilled', ascending=False).index[2:]:
        print(index)
        df2.loc[index,'jungle_flag'] = False

,matchid,participantid,totaljungleMinionsKilled,itemNames,summonerSpells,jungle_flag,num_junglers
13,NA1_3908483015,4,128,zhonya's hourglass blasting wand sorcerer's sh...,flash smite,True,3
16,NA1_3908483015,7,70,None,smite flash,True,3
11,NA1_3908483015,2,12,None,smite flash,True,3


11


,matchid,participantid,totaljungleMinionsKilled,itemNames,summonerSpells,jungle_flag,num_junglers
26,NA1_3797672185,7,160,None,smite flash,True,3
21,NA1_3797672185,2,138,thornmail dead man's plate mobility boots aegi...,smite ghost,True,3
20,NA1_3797672185,1,0,None,flash smite,True,3


20


In [95]:
df3 = df2[['matchid','participantid','jungle_flag']]
df3

,matchid,participantid,jungle_flag
0,NA1_4629886434,1,False
1,NA1_4629886434,2,True
2,NA1_4629886434,3,False
3,NA1_4629886434,4,False
4,NA1_4629886434,5,False
5,NA1_4629886434,6,False
6,NA1_4629886434,7,True
7,NA1_4629886434,8,False
8,NA1_4629886434,9,False
9,NA1_4629886434,10,False


In [ ]:
# def jungler_by_smite(df):
#         jungleFlag=[]
#         jungle=False
#         for j in range(len(df)):
            
#             if "Smite" in df.iloc[j]['all_spells']:
#                 jungle=True
#             jungleFlag.append(jungle)
#             jungle=False
#         df['jungleFlag']=jungleFlag
#         df.drop(['all_spells'], axis=1,inplace=True)

#         #list of gameIds to loop thru
#         gameIdList=list(df['gameId'].drop_duplicates())
#         #check for games with more than 2 jungles
#         for id in gameIdList:
           
#             a=df[(df['gameId'] == id) & (df['minute'] == 4) &  (df['jungleFlag']==True)]

#             for i in range(len(a)-2): #doesn't run if 2 junglers

#                 #sort junglers by jungle cs. chop off jungler   with least cs, until there are only 2 junglers
#                 a=df[(df['gameId'] == id) & (df['minute'] == 4) &  (df['jungleFlag']==True)]
#                 b=a[['jungleMinionsKilled','summonerName']].sort_values(['jungleMinionsKilled'],ascending=True)

#                 #replace jungle flag for all gameId, champion names for top index
#                 df.loc[(df['gameId'] == id) & (df['summonerName']== df.at[b.index[0],'summonerName']),'jungleFlag']=False

#         return df

In [ ]:
to_classify = help.find_to_classify(engine)
to_classify

match_id_test='NA1_4720874733'
query=f""" SELECT 
mt.matchid,
mt.minute,
mt.participantid,
mt.minionsKilled,
mt.jungleMinionsKilled,
mt.posX,
mt.posY,
md.neutralMinionsKilled as totaljungleMinionsKilled,
md.totalMinionsKilled,
lower(concat( item0Name, ' ', item1Name, ' ',  item2Name, ' ',  item3Name, ' ', 
        item4Name, ' ',  item5Name, ' ',  item6Name )) as itemNames,
lower(concat( summoner1Name,  ' ', summoner2Name )) as summonerSpells

FROM riot_api.lol_match_timeline mt
left join riot_api.lol_match_details md on mt.matchID = md.matchID and mt.participantid = md.participantid
where mt.matchId = '{match_id_test}' """
df = pd.read_sql(query, con=engine)
df